In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_pinball_loss
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
import joblib

In [4]:
df = pd.read_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/HEFTcom24/data/features.csv")

In [5]:
df = df.drop(columns=['Unnamed: 0'])   
df = df.dropna()

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=["valid_time", "Solar_MWh_credit"]), df["Solar_MWh_credit"], test_size=0.2, random_state=42, shuffle= False)

In [11]:
def objective(trial, alpha):
    
    n_estimators = trial.suggest_int("n_estimators", 100, 1000)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int("max_depth", 3, 9)
    subsample = trial.suggest_float("subsample", 0.2, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.2, 1.0)
    reg_alpha = trial.suggest_float("reg_alpha", 1e-4, 1e2)
    reg_lambda = trial.suggest_float("reg_lambda", 1e-4, 1e2)



    def custom_pinball_loss(y_true, y_pred):
        y_true = y_true.get_label() if hasattr(y_true, 'get_label') else y_true
        delta = y_pred - y_true
        grad = np.where(delta >= 0, alpha, alpha-1)
        hess = np.ones_like(y_true)  # Hessian is 1 for pinball loss
        return grad, hess

    model = XGBRegressor(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        random_state=42,
        tree_method="hist",
        objective=custom_pinball_loss,
    )

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    loss = mean_pinball_loss(y_test, y_pred, alpha)
    

    return loss


In [8]:



for alpha in alphas:


    def custom_pinball_loss(y_true, y_pred):
        y_true = y_true.get_label() if hasattr(y_true, 'get_label') else y_true
        delta = y_pred - y_true
        grad = np.where(delta >= 0, alpha, alpha-1)
        hess = np.ones_like(y_true)  # Hessian is 1 for pinball loss
        return grad, hess

NameError: name 'alphas' is not defined